In [16]:
import numpy as np

1) Samuel Janas 151927
2) Michał Skrzypek 151766

In [17]:
# set seed for reproducibility
np.random.seed(23)

In [18]:
import numpy as np

def create_transportation_problem(num_sources, num_destinations, zero_chance=0.1, M_chance=0.05, M_value=999):
    """
    Creates a transportation problem with specified number of sources and destinations.
    Costs can be randomly set to zero or a very high number (M_value) with given chances.
    
    Parameters:
    num_sources (int): Number of sources.
    num_destinations (int): Number of destinations.
    zero_chance (float): Probability of a cost being zero.
    M_chance (float): Probability of a cost being M_value (very high cost).
    M_value (int): The very high cost to use for prohibitive connections.
    
    Returns:
    tuple: Returns a tuple containing the cost matrix, supply, and demand arrays.
    """
    
    if num_sources <= 0 or num_destinations <= 0:
        raise ValueError("Number of sources and destinations must be positive integers")
    
    # Random cost matrix with occasional zeros
    cost_matrix = np.random.randint(1, 11, size=(num_sources, num_destinations))
    zero_mask = np.random.rand(num_sources, num_destinations) < zero_chance
    M_mask = np.random.rand(num_sources, num_destinations) < M_chance
    cost_matrix[zero_mask] = 0
    cost_matrix[M_mask] = M_value

    # Random supply and demand ensuring total supply equals total demand
    total_units = np.random.randint(20, 50) * num_sources
    supply = np.random.randint(1, total_units // num_sources, size=num_sources)
    demand = np.random.randint(1, total_units // num_destinations, size=num_destinations)
    
    # Adjust supply and demand to introduce occasional zero rows/columns
    if np.random.rand() < zero_chance:
        supply[np.random.randint(0, num_sources)] = 0
    if np.random.rand() < zero_chance:
        demand[np.random.randint(0, num_destinations)] = 0
    
    # Ensure total supply equals total demand after zero introduction
    supply_demand_difference = supply.sum() - demand.sum()
    if supply_demand_difference > 0:
        demand[-1] += supply_demand_difference
    else:
        supply[-1] -= supply_demand_difference

    return cost_matrix, supply, demand

In [19]:
def get_entering_variable_position(ws):
    # Create a copy of ws and sort it based on the second element of each tuple
    ws_copy = ws.copy()
    ws_copy.sort(key=lambda w: w[1])
    # Return the first element of the last tuple in the sorted list
    return ws_copy[-1][0]

In [20]:
def can_be_improved(ws):
    for p, v in ws:
        if v > 0: return True
    return False


In [21]:
def get_ws(bfs, costs, us, vs):
    ws = []
    # Iterate through the costs matrix
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            # Check if (i, j) is a non-basic variable
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                # Calculate and append the opportunity cost
                ws.append(((i, j), us[i] + vs[j] - cost))
    return ws


In [22]:
def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    # Get all unvisited nodes in the same row or column as the last node
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    # Return the appropriate nodes based on the length of the loop
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        return nodes_in_column if row_move else nodes_in_row

In [23]:
def loop_pivoting(bfs, loop):
    # Separate the loop into even and odd positions
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    # Determine the position to leave from the loop
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    # Adjust the values of basic variables based on the leaving value
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs


In [24]:
def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            # Check if the loop can be closed
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        # Recursively build the loop
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_position])

In [25]:
def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    # Calculate the dual variables u and v
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: continue
                
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
            
    return us, vs   

In [26]:
def north_west_corner(supply, demand):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    i = 0
    j = 0
    bfs = [] # basic feasible solution
    # Assign supply to demand starting from the top-left corner
    while len(bfs) < len(supply) + len(demand) - 1:
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s, d)
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i, j), v))
        # Move to the next supply or demand point
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
    return bfs 

In [33]:
def transportation_simplex_method(supply, demand, costs, penalties=None):
    # The main function for the transportation simplex method
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, costs)
        ws = get_ws(bfs, costs, us, vs)
        # Recursively find and pivot the entering and leaving variables
        if can_be_improved(ws):
            ev_position = get_entering_variable_position(ws)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))
        return bfs
    
    basic_variables = inner(north_west_corner(supply, demand))
    solution = np.zeros((len(costs), len(costs[0])))
    # Construct the final solution
    for (i, j), v in basic_variables:
        solution[i][j] = v

    return solution

In [35]:
cost_matrix, supply, demand = create_transportation_problem(20, 20)

In [36]:
solution = transportation_simplex_method(supply, demand, cost_matrix)

18
14
20
18
24
22
22
22
20
22
22
20
20
18
18
18
16
18
16
14
8
8
8
6
8
6
4
4
8
12
16
12
12
12
6
12
12
14
14
6
8
8
4
6
10
8
8
14
10
12
12
18
4


In [30]:
def get_total_cost(costs, solution):
    total_cost = 0
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            total_cost += cost * solution[i][j]
    return total_cost

In [31]:
print("Total cost:", get_total_cost(cost_matrix, solution))

Total cost: 103.0


In [32]:
import pulp

def solve_with_pulp(cost_matrix, supply, demand):
    num_sources, num_destinations = cost_matrix.shape
    problem = pulp.LpProblem("Transportation_Problem", pulp.LpMinimize)

    # Create variables for flow on transportation arcs
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_sources) for j in range(num_destinations)),
                              lowBound=0)

    # Objective function: Minimize total transportation cost
    problem += pulp.lpSum([cost_matrix[i][j] * x[(i, j)] for i in range(num_sources) for j in range(num_destinations)])

    # Constraints for supply
    for i in range(num_sources):
        problem += pulp.lpSum([x[(i, j)] for j in range(num_destinations)]) == supply[i]

    # Constraints for demand
    for j in range(num_destinations):
        problem += pulp.lpSum([x[(i, j)] for i in range(num_sources)]) == demand[j]

    # Solve the problem
    problem.solve()

    # Return the objective function value and the solution (if it's found)
    if problem.status == pulp.LpStatusOptimal:
        solution = {(i, j): x[(i, j)].varValue for i in range(num_sources) for j in range(num_destinations)}
        return pulp.value(problem.objective), solution
    else:
        return None, None

# Solve using PuLP and compare with your solution
pulp_cost, pulp_solution = solve_with_pulp(cost_matrix, supply, demand)

# Check if the total costs are the same
print("PuLP solution total cost:", pulp_cost)

PuLP solution total cost: 103.0
